In [12]:
##Import all packages
from __future__ import print_function
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
from numpy import zeros, arange
from scipy.integrate import odeint
from math import pi, log10, e, log
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd

In [13]:
def lst(min, max, length):
  lists = []
  for i in range(length):
    lists.append(min+(max-min)*i/length)
  return lists

In [14]:
##Equation of state that returns density based on the pressure
def eos(p, K, g):
  rho = (p/K)**(1/g) + p/(g-1)
  return rho

In [15]:
def piece(p):
  global minP
  global maxP
  global pieces
  global gammaMin
  global gammaMax

  ps = lst(minP, maxP, pieces)
  gs = lst(gammaMin, gammaMax, pieces)

  k_i = [1]

  for i in range(len(gs)-1):
    k_i.append(k_i[i]*ps[i]**(gs[i]-gs[i+1]))

  for i in range(len(ps)):
    if p < e**ps[i]:
      return(eos(p, k_i[i], gs[i]))
  return(eos(p, k_i[-1], gammaMax))


In [16]:
minP = None
maxP = None
pieces = None
gammaMin = None
gammaMax = None

In [17]:
##Completes the TOV equations
def tov(vars, radius):
  p = vars[0]
  m = vars[1]
  d = piece(vars[0])
  dPdr = -1*(((d+p)*(m+(4*pi*p*radius**3)))/(radius*(radius-(2*m))))
  dmdr = 4*pi*d*radius**2
  return dPdr, dmdr

In [18]:
##Integrates the system of differential equations
##  and calculates the maximum stable mass and radius based on pressure
def tovSolve(pres, sP, bP, pie, sG, bG):

  global minP
  global maxP
  global pieces
  global gammaMin
  global gammaMax

  minP = sP
  maxP = bP
  pieces = pie
  gammaMin = sG
  gammaMax = bG

  pressureCutoff = 1e-13*pres

  startingRadius =1e-5
  finalRadius = 4
  step = 0.0001

  ans = odeint(tov, [pres, 0],arange(startingRadius, finalRadius, step), printmessg=1)

  radii = arange(startingRadius, finalRadius, step)
  masses = ans[:,1]
  pressures = ans[:,0]

  count = 0
  mass = 0.0
  radius = startingRadius

  for i in pressures:
    if i > pressureCutoff:
      count += 1
  radius = (count-1)*step
  mass = masses[count-1]

  return radius, mass

In [19]:
##Sets up the data for the Mass VS Radius plot using varying central density
def mass_radius(minimumPresLn, maximumPresLn, step, sP, bP, pie, sG, bG):
  pres = zeros(step)
  mass = zeros(step)
  radius = zeros(step)
  dens = zeros(step)
  for i in range(step):
    pres[i] = minimumPresLn + (maximumPresLn-minimumPresLn)*i/step
    radius[i],mass[i] = tovSolve(e**pres[i], sP, bP, pie, sG, bG)
  for m in range(len(mass)):
    mass[m]*=8.54
  for r in range(len(radius)):
    radius[r]*=12.24

  for p in range(len(pres)):
    dens[p] = log(piece(e**pres[p]))

  return radius, mass, pres, dens

In [20]:
ms =[]
rs = []
def series(sP, bP, pie, sG, bG):
    global ms
    global rs

    ans = mass_radius(-12, 3, 20, sP, bP, pie, sG, bG)
    ms.append(list(ans[1]))
    rs.append(list(ans[0]))
    for i in range(len(rs)):
      plt.scatter(rs[i], ms[i])
      plt.xlabel('Radius (Km)')
      plt.ylabel('Mass (Solar Masses)')
w = interact(series, sP=(-12, -8, 0.25), bP=(0, 5, 0.25), pie=(5,100,1), sG=(1.9, 2, 0.01), bG=(2.01, 2.1, 0.01));

interactive(children=(FloatSlider(value=-10.0, description='sP', max=-8.0, min=-12.0, step=0.25), FloatSlider(…

In [21]:
import random

rows = []
for k in range(25):
  sP = random.random()*-4-8
  bP = random.random()*5
  pie = int(random.random()*95+5)
  sG = random.random()*0.1+1.9
  bG = random.random()*0.1+2.01

  ans = mass_radius(-12, 3, 20, sP, bP, pie, sG, bG)

  l = list(ans[0]) + list(ans[1])
  l.append(sP)
  l.append(bP)
  l.append(pie)
  l.append(sG)
  l.append(bG)
  rows.append(l)

  print(k)

C:\Users\kalha\AppData\Local\Temp\ipykernel_6908\3407550356.py:3: RuntimeWarning: invalid value encountered in scalar power
  rho = (p/K)**(1/g) + p/(g-1)
C:\Users\kalha\AppData\Local\Temp\ipykernel_6908\3481544681.py:23: ODEintWarning: Excess accuracy requested (tolerances too small). Run with full_output = 1 to get quantitative information.
  ans = odeint(tov, [pres, 0],arange(startingRadius, finalRadius, step), printmessg=1)
C:\Users\kalha\AppData\Local\Temp\ipykernel_6908\622765697.py:16: ComplexWarning: Casting complex values to real discards the imaginary part
  dens[p] = log(piece(e**pres[p]))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [ ]:
import pandas as pd

data = rows
column = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI', 'AJ', 'AK', 'AL', 'AM', 'AN', 'AO', 'AP', 'AQ', 'AR', 'AS']
df = pd.DataFrame(data, columns=column)

df.to_csv(r'NeutronCurveTrainer.csv', index = False)